In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools


In [18]:
df = pd.read_csv("ETHUSDT_4h_from_1_Jan_2020.csv")

In [42]:
# === ПАРАМЕТРЫ ===
impulse_threshold = 10.5      # % падения
target_profit_pct = 1.2    # take profit
stop_loss_pct = 99          # stop loss
window = 100                # свечей в анализе

# === ЗАГРУЗКА ДАННЫХ ===
# Убедись, что df содержит: 'open_time', 'high', 'low', 'close'
df["open_time"] = pd.to_datetime(df["open_time"], utc=True)

highs = df["high"].to_numpy(dtype=np.float64)
lows = df["low"].to_numpy(dtype=np.float64)
closes = df["close"].to_numpy(dtype=np.float64)
times = df["open_time"].values.astype("datetime64[ns]")

# === ВЫЧИСЛЕНИЕ ИМПУЛЬСОВ ===
rolling_max = pd.Series(closes).rolling(window=window).max().to_numpy()
rolling_min = pd.Series(closes).rolling(window=window).min().to_numpy()
drawdowns = (rolling_min / rolling_max - 1) * 100
impulse_mask = drawdowns <= -impulse_threshold

# === БЭКТЕСТ С ПРОГРЕССБАРОМ ===
in_trade = False
results = []

i = 0
n = len(closes)
pbar = tqdm(total=n, desc="🚀 Backtesting")

while i < n:
    if not in_trade and impulse_mask[i]:
        entry_price = closes[i]
        entry_time = times[i]
        take_profit = entry_price * (1 + target_profit_pct / 100)
        stop_loss = entry_price * (1 - stop_loss_pct / 100)
        in_trade = True

        j = i + 1
        while j < n:
            if highs[j] >= take_profit:
                exit_price = take_profit
                exit_time = times[j]
                exit_type = "take_profit"
                break
            elif lows[j] <= stop_loss:
                exit_price = stop_loss
                exit_time = times[j]
                exit_type = "stop_loss"
                break
            j += 1
        else:
            print("⚠️ Неисполненная сделка:", times[i])
            break

        duration = (exit_time - entry_time) / np.timedelta64(1, "m")
        profit_pct = (exit_price / entry_price - 1) * 100

        results.append({
            "entry_time": entry_time,
            "entry_price": entry_price,
            "exit_time": exit_time,
            "exit_price": exit_price,
            "exit_type": exit_type,
            "duration": duration,
            "profit_pct": profit_pct,
            "stop_loss": stop_loss,
            "stopped_out": exit_type == "stop_loss"
        })

        in_trade = False
        pbar.update(j - i)
        i = j
    else:
        i += 1
        pbar.update(1)

pbar.close()

# === СТАТИСТИКА ===
result_df = pd.DataFrame(results)

print("📈 Сделок:", len(result_df))
print("✅ Прибыльных:", (result_df["exit_type"] == "take_profit").sum())
print("💥 Стопов:", result_df["stopped_out"].sum())
print("🕒 Среднее время (мин):", result_df["duration"].max())


start_balance = 10
balance = 10
for p in result_df["profit_pct"]:
    balance *= (1 + p / 100)
print(f"💰 Финальный капитал: ${balance:.2f}")
profit_total_pct = (balance / start_balance - 1) * 100
print(f"📊 Общая доходность: {profit_total_pct:.2f}%")

🚀 Backtesting: 100%|██████████| 2809229/2809229 [00:01<00:00, 1976748.69it/s]

📈 Сделок: 137
✅ Прибыльных: 137
💥 Стопов: 0
🕒 Среднее время (мин): 857444.0
💰 Финальный капитал: $51.25
📊 Общая доходность: 412.54%


In [19]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools

# Параметры
impulse_range = [5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 20, 25]
tp_range = [0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 9, 10]
window_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 20, 25]
stop_loss_pct = 99  # По сути отключён

results = []

# Подготовка
df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
highs = df["high"].to_numpy(dtype=np.float64)
lows = df["low"].to_numpy(dtype=np.float64)
closes = df["close"].to_numpy(dtype=np.float64)
times = df["open_time"].values.astype("datetime64[ns]")

# Список всех комбинаций
combos = list(itertools.product(impulse_range, tp_range, window_range))
pbar = tqdm(total=len(combos), desc="🔄 Grid Search")

# Перебор
for impulse_threshold, tp, window in combos:
    rolling_max = pd.Series(closes).rolling(window=window).max().to_numpy()
    rolling_min = pd.Series(closes).rolling(window=window).min().to_numpy()
    drawdowns = (rolling_min / rolling_max - 1) * 100
    impulse_mask = drawdowns <= -impulse_threshold

    in_trade = False
    balance = 10
    i = 0
    n = len(closes)
    durations = []
    entry_times = []

    while i < n:
        if not in_trade and impulse_mask[i]:
            entry_price = closes[i]
            entry_time = times[i]
            take_profit = entry_price * (1 + tp / 100)
            stop_loss = entry_price * (1 - stop_loss_pct / 100)
            in_trade = True

            j = i + 1
            while j < n:
                if highs[j] >= take_profit:
                    exit_price = take_profit
                    exit_time = times[j]
                    break
                elif lows[j] <= stop_loss:
                    exit_price = stop_loss
                    exit_time = times[j]
                    break
                j += 1
            else:
                break

            profit_pct = (exit_price / entry_price - 1) * 100
            balance *= (1 + profit_pct / 100)

            duration = (exit_time - entry_time) / np.timedelta64(1, "m")
            durations.append(duration)
            entry_times.append(entry_time)

            in_trade = False
            i = j
        else:
            i += 1

    total_pct = (balance / 10 - 1) * 100

    if durations:
        d_min = np.min(durations)
        d_max = np.max(durations)
        d_mean = np.mean(durations)
        d_median = np.median(durations)
        last_entry = max(entry_times)
    else:
        d_min = d_max = d_mean = d_median = 0
        last_entry = pd.NaT

    results.append({
        "impulse_threshold": impulse_threshold,
        "take_profit": tp,
        "window": window,
        "profit_total_pct": total_pct,
        "duration_min": d_min,
        "duration_max": d_max,
        "duration_mean": d_mean,
        "duration_median": d_median,
        "last_entry_time": last_entry
    })

    pbar.update(1)

pbar.close()
grid_df = pd.DataFrame(results)
grid_df = grid_df.sort_values("profit_total_pct", ascending=False)

# ТОП-10
print("🏆 ТОП-10 лучших комбинаций:")
grid_df.head(10)


🔄 Grid Search: 100%|██████████| 4290/4290 [00:26<00:00, 163.10it/s]

🏆 ТОП-10 лучших комбинаций:


,impulse_threshold,take_profit,window,profit_total_pct,duration_min,duration_max,duration_mean,duration_median,last_entry_time
1781,10,1.5,13,5553.240343,240.0,244800.0,5190.553506,480.0,2025-02-02 00:00:00
2831,13,3.0,13,5150.348248,240.0,155280.0,6485.373134,720.0,2024-12-20 16:00:00
2500,12,3.0,12,5150.348248,240.0,227520.0,8502.089552,720.0,2024-12-21 04:00:00
1175,8,3.0,6,4704.812408,240.0,335040.0,11728.854962,960.0,2025-02-03 12:00:00
2485,12,2.5,12,4608.938312,240.0,227520.0,7163.076923,480.0,2024-12-21 04:00:00
293,5,8.0,9,4590.161251,240.0,248160.0,19809.600000,4680.0,2021-11-16 00:00:00
292,5,8.0,8,4590.161251,240.0,248160.0,19809.600000,4680.0,2021-11-16 00:00:00
294,5,8.0,10,4590.161251,240.0,248160.0,19838.400000,4680.0,2021-11-16 00:00:00
596,6,6.0,13,4579.366994,240.0,251760.0,15007.272727,2040.0,2021-11-16 00:00:00
989,7,10.0,25,4425.925557,480.0,246720.0,24174.000000,7080.0,2021-10-29 00:00:00


In [20]:
grid_df.head(10)

,impulse_threshold,take_profit,window,profit_total_pct,duration_min,duration_max,duration_mean,duration_median,last_entry_time
1781,10,1.5,13,5553.240343,240.0,244800.0,5190.553506,480.0,2025-02-02 00:00:00
2831,13,3.0,13,5150.348248,240.0,155280.0,6485.373134,720.0,2024-12-20 16:00:00
2500,12,3.0,12,5150.348248,240.0,227520.0,8502.089552,720.0,2024-12-21 04:00:00
1175,8,3.0,6,4704.812408,240.0,335040.0,11728.854962,960.0,2025-02-03 12:00:00
2485,12,2.5,12,4608.938312,240.0,227520.0,7163.076923,480.0,2024-12-21 04:00:00
293,5,8.0,9,4590.161251,240.0,248160.0,19809.600000,4680.0,2021-11-16 00:00:00
292,5,8.0,8,4590.161251,240.0,248160.0,19809.600000,4680.0,2021-11-16 00:00:00
294,5,8.0,10,4590.161251,240.0,248160.0,19838.400000,4680.0,2021-11-16 00:00:00
596,6,6.0,13,4579.366994,240.0,251760.0,15007.272727,2040.0,2021-11-16 00:00:00
989,7,10.0,25,4425.925557,480.0,246720.0,24174.000000,7080.0,2021-10-29 00:00:00


In [21]:
# === СОХРАНЕНИЕ CSV ===
symbol = "ETHUSDT"         # <-- замени на актуальную монету
interval = "4h"            # <-- и таймфрейм (1m, 5m, 1h и т.д.)
combo_count = len(combos)

filename = f"optimization_{symbol}_{interval}_{combo_count}_combos.csv"
grid_df.to_csv(filename, index=False)

print(f"💾 Результаты сохранены: {filename}")


💾 Результаты сохранены: optimization_ETHUSDT_4h_4290_combos.csv
